![NERLab](http://www.fee.unicamp.br/sites/default/files/docentes/leoelias/imagens/logo_v1%28WEB%29.png)
--

# NeMu: A neuromuscular model notebook

[Ricardo Gonçalves Molinari](mailto:molinari@gmail.com) and [Leonardo Abdala Elias](mailto:leoelias@unicamp.br)


- Version 0.1 | Last update: Jan. 6, 2019

- Version 0.2 | Last update: July 22, 2019

- Version 0.3 | Last update: October 3, 2019

- Version 0.4 | Last update: July 18, 2020

***

# Import Libraries and Create Model Objects

In [1]:
from nerlab import fug, emg, force, util
import ipywidgets as wi

util.config_plots(style = 'seaborn-whitegrid', font_size =20)
mnpool = fug.Phemo()
muscle_emg = emg.Emg_mod(mnpool)
muscle_force = force.Muscle_force(mnpool)
sim = util.sim_results(mnpool, muscle_emg, muscle_force)

***

# The Motor Pool Model

## Recruitment pattern of motor units

This section of the model generates the static input-output relationship of the motor pool. Here, we adopted the phenomenological model by [Fuglevand et al. (1993)](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools"), where the common drive to the motor neuron pool is mapped to the motor unit (MU) firing rate. A key aspect in this mapping is the recruitment pattern of the MUs.

As well known, each MU encompasses a single motor neuron and its innervated muscle fibers. In this model, three types of MUs were represented:
- Type I (or S) MUs: low-threshold MUs with slowly contracting muscle units;
- Type IIa (or FR) MUs: these MUs have a higher recruitment threshold (as compared to type I MUs) and its muscle units are fast contracting but fatigue resistant;
- Type IIb (or FF) MUs: these MUs have the highest recruitment threshold with fast contracting and fatigable muscle units.

### Recruitment range

***Recruitment Range (RR)*** is used to define the *recruitment threshold excitation (RTE)* of each MU.

$$RTE[i]  =exp \left ( \frac{ln(RR)i}{n} \right )$$
    
where:
- $i$ is the MU index;
- $n$ is the total number of MUs.

### Relationship between MU firing rate and the excitatory drive

The following equation is used to define the *firing rate (FR)* of each MU as a function of the *excitatory (common) drive*.

$$\begin{align}
FR_i[t] & = g_e(E[t]-RTE_i) + MFR  \, , E[t] \ge RTE_i \\
PFR_i & = PFR_1 - PFRD \frac{RTE_i}{RTE_n} \\
\end{align}$$

where:
- $FR_i[t]$ is the mean firing rate of MU $i$ at time $t$; 
- $g_e$ is the gain of the relationship between the MU firing rate and the excitatory drive;
- $E[t]$ is the excitatory drive (or common drive) as function of time;
- $MFR$ is the minimum firing rate.
- $PFR_i$ is the peak firing rate of MU $i$;
- $PFRD$ is the peak firing rate difference between the first and the last MU of the pool;

For more information about the physiology of MUs, please refer to the monograph by [Kernell (2006)](https://doi.org/10.1093/acprof:oso/9780198526551.001.0001 "The Motoneurone and its Muscle Fibres").

In [2]:
ui1,ws1 = util.wi1()
out1 = wi.interactive_output(mnpool.view_organization, ws1)
display(wi.HBox([ui1,out1]))

## Definition of the excitatory (common) drive

The common drive and the recruitment pattern of the MUs (defined above) are used to generate the MU spike trains. These spike trains will command the muscle units so as to produce muscle force and electromyogram (EMG).

Different force intesities (percentage of the maximum force) may be explored with ***Trapezoidal*** or ***Sinusoidal*** waveform commands. To simulate a steady isometric contraction, you can choose the ***Trapezoidal*** waveform, and then define ***Plateau On*** as zero and ***Plateau Off*** as the final simulation time.

MU recruitment and de-recruitment can be observed using a ***Trapezoidal*** command, but in this scenario the value of the parameter ***Onset*** should be less than the ***Plateau On***, and ***Offset*** should be less than ***Plateau Off***.

Periodic contractions can be simulated by changing the command to the ***Sinusoidal*** waveform, with the following parameters: ***Peak Intensity*** and ***Frequency***.

In [3]:
ws2,ui2 = util.wi2()
out2 = wi.interactive_output(mnpool.view_excitatory,ws2)
display(wi.HBox([ui2,out2]))

## Motor unit spike trains

The MU activity (spike trains) resembles a stochastic point process with a gaussian distribution. The model by [Fuglevand et al. (1993)](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools") simulates the spike times as:

$$ t_{i,j}= \mu + \sigma. Z + t_{i,j-1}$$

where:
- $t_{i,j}$ is the $j$ spike time of the MU $i$;
- $\mu = 1/FR_{ij}$ is the mean interspike interval;
- $\sigma$ is the standard deviation of each interspike interval;
- $Z$ represents how the spike time deviates from the mean value (a random value between -3.9 and 3.9).  

The standard deviation ($\sigma$) is defined as function of the ***coeficient of variation ($CoV$)*** and the mean spike interval ($\mu$), as $\sigma = \mu . CoV$.

### Motor unit synchronization

Convergent presynaptic commands to the spinal motor neurons promote common fluctuations on the membrane potentials of these cells. These common sources increases the probability of synchronous discharges of MUs (the so-called MU synchronization).

To simulate this phenomenon, we adopted the model by [Yao et al. (2000)](https://doi.org/10.1152/jn.2000.83.1.441 "Motor-Unit Synchronization Increases EMG Amplitude and Decreases Force Steadiness of Simulated Contractions"). However, an extra step was added to the algorithm in order to deal with time varying excitations (trapezoidal and sinusoidal).

The ***Synchronization Level [%]*** operates as follows: 
1. A percentage of MUs will be randomly selected and they will be the references for the adjustments of MU spike times;
2. Also, as the percentage of selected MU spike times that will serve as reference for adjustment;
3. Also, as the percentage of recruited motor units of the pool that will have their discharge times adjusted with the reference discharge time.
4. And finally, some deviation (***Synchronism Standard Deviation [ms]***) across the synchronized discharge times where added to better represent this phenomenon.

In [4]:
wi3 = util.wi3()
_ = wi.interact_manual(mnpool.view_neural_command, cv_factor = wi3[2], CV_init = wi3[0],
                       CV_final = wi3[1], synch_level = wi3[3], sigma = wi3[4])

interactive(children=(FloatSlider(value=1.5, description='CoV variation factor (use zero for cte):', layout=La…

***

# Surface Electromyogram (sEMG)

The *surface electromyogram (sEMG)* emerges from the electric activity of the muscle fibres during a given contraction. Currents flowing across the sarcolemma (as a result of the muscle fiber action potential) produce an electric potential that can be recorded from extracellular electrodes. The recorded extracellular potentials (produced by all recruited MUs) are spatially filtered by the volume conductor, which includes the muscle, adipose, and skin tissues. It is worth noting that the MU territories are sparsely distributed in the muscle cross-section. Here, we represented the sEMG recorded by bipolar electrodes (*red star* in the following graph) located on the skin.

## Muscle cross-section morphology

Four different morphologies for the muscle cross-section were included to represent the desired muscle. The following morphologies were included: circle, ring, pizza, and ellipse. Also, cross-section area (***CSA***), fat thickness (***Fat Layer [mm]***), and skin thickness (***Skin Layer [mm]***) are properties to be adjusted. For the ring, pizza, and ellipse morphologies you need to adjust two additional parameters:
- For the ***ring*** morphology, ***Proportion*** defines the ratio between internal and external muscle radii, while for the ***ellipse*** the referred parameter defines the ratio between the semi-minor and semi-major axes;
- ***Theta [rad]*** defines the angle formed between the vertical line (which passes through the origin) and the line that defines the lateral boundary of the muscle CSA, which also passes through the origin.

In [5]:
ws4,ui4 = util.wi4()
out4 = wi.interactive_output(muscle_emg.view_morpho, ws4)
display(wi.HBox([ui4, out4]))

## Motor unit distribution within the muscle CSA

Many human muscles present some level of MU regionalization within its CSA ([Johnson et al., 1973](https://doi.org/10.1016/0022-510X(73)90023-3 "Data on the distribution of fibre types in thirty-six human muscles")).

To simulate the MU regionalization, we define two different distributions, one for the type I MUs and another for type II MUs. Each MU position is defined by polar coordanates (i.e., radius and angle). All motor units will have their polar angle defined by a uniformly distributed random variable with the range depending on the choosen morphology. The radius is also defined by a random variable, but with a gaussian distribution.

Here we can control the radius with the parameter ***Type I $\mu$***, which defines the center of the distribution for Type I MUs (normalized by the muscle external radius). Similarly, ***Type II $\mu$*** defines the center of the distribution for Type II (a and b) MUs. The parameter ***Type I $\sigma$*** defines the standard deviation of the gaussian distribution of the radius for Type I MUs (also normalized by the muscle external radius), while ***Type II $\sigma$*** is the standard deviation of the radius distribution for Type II (a and b) MUs.

### Innervation ratio

The size of MU territory is highly correlated with the number of muscle fibres innervated by a single motor neuron, which varies across the muscles (see [Enoka and Fuglevand, 2001](https://doi.org/10.1002/1097-4598(200101)24:1<4::AID-MUS13>3.0.CO;2-F "Motor unit physiology: Some unresolved issues")). A typical skeletal muscle comprises many MUs with relatively few muscle fibres and few MUs with a large quantity of muscle fibres. This MU innervation ratio across the pool can be modelled by:

$$y_i = y_1.exp \left (ln(R)\frac{i}{n} \right )$$

where:
- $y_i$ is the innervation number of the MU $i$;
- $y_1$ is the innervation number of the smallest MU;
- $R$ is the ***innervation number ratio*** between the largest ($y_n$) and smallest ($y_1$) MU.

In [6]:
ui5,ws5 = util.wi5()
out5 = wi.interactive_output(muscle_emg.view_distribution, ws5)
display(wi.VBox([ui5, out5]))

## Motor unit action potentials (MUAPs)

Motor unit action potentials (MUAP) recorded from bipolar surface electrodes can be represented by ***1st or 2nd order*** Hermite-Rodriguez (HR) functions ([Lo Conte et al., 1994](http://doi.org/10.1109/10.335863 "Hermite Expansions of Compact Support Waveforms: Applications to Myoelectric Signals")). These functions fit well the MUAP shape and has been used in previous studies. The ***1st order*** HR function models biphasic MUAPs while the ***2nd order*** function models triphasic MUAPs. In the present model, a given HR function (1st or 2nd order) is randomly attributed (uniform distribution) to a given MU.

These functions are defined by the following equations:

$$\begin{align}
HR_1(t) &= \frac{A_M.(t-t_{AP})}{\lambda_M} e^{  - \left (\frac{t-t_{AP}}{\lambda_M} \right )^2}u(t-t_{AP}) \\
HR_2(t) &= A_M \left [1 - 2 \left ( \frac{t-t_{AP}}{\lambda_M} \right )^2 \right ] e^{- \left (\frac{t-t_{AP}}{\lambda_M} \right )^2} u(t-t_{AP})
\end{align}$$

where:
- $HR_i$ is the $i$-th order HR function;
- $A_M$ is the amplitude factor;
- $\lambda_M$ is the duration factor;
- $t_{AP}$ is the MU discharge time;
- $u(t)$ is the Heaviside function;

Distinct amplitude and duration factors are attributed to each MU. To define each MUAP amplitude factor, an exponential interpolation between the ***First MUAP Amplitude [mV]*** and the ***Last MUAP Amplitude [mV]*** is used. The same method is adopted to define the duration factor of each MUAP.

In [7]:
ui6,ws6 = util.wi6()
out6 = wi.interactive_output(muscle_emg.view_muap, ws6)
display(ui6, out6)

Output()

## Volume conductor (spatial filtering)

The isotropic filtering property of the volume conductor was modelled as a function of the distance between the location of the motor unit territory in the muscle CSA and the surface electrodes. We adopted the model proposed by [Cisi and Kohn (2008)](http://doi.org/10.1007/s10827-008-0092-8 "Simulation system of spinal cord motor nuclei and associated nerves and muscles, in a Web-based architecture").

$$\begin{align}
V &= V_0. e^{ \left ( \frac{-d}{\tau_{at}} \right )} \\
T &= T_0.(1+C.d)
\end{align} $$

where:
- $V$ is the MUAP amplitude at the skin surface;
- $V_0$ is the MUAP amplitude at the center of the MU territory;
- $d$ is the distance between the center of the MU territory and the center of bipolar electrode;
- $\tau_{at}$ is the ***Amplitude Attenuation Factor [$mm^{-1}$]***;
- $T$ is the MUAP duration at the electrode;
- $T_0$ is the MUAP duration at the center of the MU territory;
- $C$ is the ***Widening Factor [$mm^{-1}$]***.

In [8]:
ui7,ws7 = util.wi7()        
out7= wi.interactive_output(muscle_emg.view_attenuation,ws7)
display(ui7,out7)

Output()

## Generation of sEMG

Finally, the sEMG signal generated by the muscle is modelled as the summed activity of all trains of MUAPs.

Some noise is always present in experimental recordings of sEMG, and this measurement noise was modelled as an additive white gaussian noise with amplitude defined by the parameter ***Noise Level (Standard deviation)[mV]***. 

Also, sEMG equipment encompasses a bandpass filter. Here, we adopted a 4th-order bandpass Butterworth filter with cutoff frequencies adjusted by the parameters ***Highpass Cutoff Frequency [Hz]*** and ***Lowpass Cutoff Frequency [Hz]***.

In [9]:
wi8 = util.wi8()
_ = wi.interact_manual(muscle_emg.view_semg, add_noise = wi8[0], noise_level = wi8[1], 
                       add_filter = wi8[4], bplc = wi8[2], bphc = wi8[3])

interactive(children=(Checkbox(value=True, description='Add Noise', layout=Layout(width='400px'), style=Descri…

## Analysis of sEMG

To analyse the sEMG signal in time domain, which can be non-stationary, we implemented a moving RMS, which is an estimate of the time varying EMG amplitude [(De Luca, 1997)](https://doi.org/10.1123/jab.13.2.135 "The Use of Surface Electromyography in Biomechanics"). Furthermore, to analyse the EMG signal in frequency domain, we used two methods: the Welch's periodogram ([Welch, 1967](https://doi.org/10.1109/TAU.1967.1161901 "The use of fast Fourier transform for the estimation of power spectra: A method based on time averaging over short, modified periodograms")) and the spectrogram.

Welch's method is an approach for estimating the power spectral density (PSD) of a random process. This method aims at reducing the estimation error of the PSD at the cost of decreasing the spectral resolution. The classical Welch's method is more adequate for stationary signals. Conversely, the spectrogram uses the Short-Time Fourier Transform (STFT) to evaluate the spectral content of the signal as a function of the time, and it is suitable for non-stationary (time-varying) signals.

Still in the following cell, one can visualize the contribution of each MUAP train (after filtering of volume conductor) for the sEMG signal. You can choose the MU by selecting the ***motor unit index #***.

In [11]:
ui9,ws9 = util.wi9(muscle_emg)
out9 = wi.interactive_output(muscle_emg.analyses, ws9)
display(ui9, out9)

Output()

***

# Muscle Force

## Motor unit twitch

MU twitch is the mechanical response of the muscle fibres to an action potential produced by the innervating motor neurons. It can be modelled as the impulse response of a critically damped second-order system ([Fuglevand et al., 1993](https://doi.org/10.1152/jn.1993.70.6.2470 "Models of recruitment and rate coding organization in motor-unit pools")).

The *peak twitch force* generated by each MU of the pool is modeled by the following equation:

$$ P[i] = P_0.e^{\frac{ln(RP).i}{n}}$$

where:
- $P[i]$ is the peak twitch force of MU $i$;
- $P_0$ is the peak twitch force of the first MU (***First MU Twitch Amplitude [mN]***);
- $RP$ is the the range of twitch forces (***Range of MU Twitches***).

The *contraction time* represents the time between the action potential arrival at the muscle unit and the peak twitch force. This parameter varies across the pool according to the following equation:

$$ T[i]=T_L \left ( \frac{1}{P_i} \right )^{1/log_{RT}RP} $$

where:
- $T[i]$ is the contraction time of MU $i$;
- $T_L$ is the largest contraction time (***Maximum Contraction Time [ms]***), which is associated to the first MU recruited;
- $RT$ is the ***Range of Contraction Times***.

In [12]:
ui10,ws10 = util.wi10()
out10 = wi.interactive_output(muscle_force.view_mu_twitch, ws10)
display(ui10,out10)

Output()

## Motor unit tetanic force and saturation

In [13]:
ui11,ws11 = util.wi11(muscle_force)
out11 = wi.interactive_output(muscle_force.view_saturation, ws11)
display(ui11, out11)

Output()

## Force Generation

The *muscle force* is modelled as the linear sum of the forces produced by all recruited MUs. Each MU force is generated by the convolution between the MU twitch and the MU spike train, as proposed by [Cisi and Kohn (2008)](http://doi.org/10.1007/s10827-008-0092-8 "Simulation system of spinal cord motor nuclei and associated nerves and muscles, in a Web-based architecture").

In [14]:
wif = wi.interact_manual(muscle_force.gen_force)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

## Force Analysis

In [16]:
ui12, ws12 = util.wi12(muscle_force)
out12 = wi.interactive_output(muscle_force.analyses, ws12)
display(ui12, out12)

Output()

***

# Saving Simulation Results

The configuration used to simulate this neuromuscular system and its results can be saved in the following folder: `\simulation_results\folder_name`. The  *folder name* can be selected in the following cell. To save it, just click on ***Run interact*** button.


In [7]:
ws13 = util.wi13()
_ = wi.interact_manual(sim.save_results, save_conf = ws13[0], save_emg = ws13[2], 
                          save_force = ws13[3], folder_name = ws13[4], save_spikes = ws13[1])

interactive(children=(Checkbox(value=True, description='Save Simulation Config.'), Checkbox(value=True, descri…